<a href="https://colab.research.google.com/github/dlmacedo/deep-learning-class/blob/master/notebooks/StackOver_Dataset_Classification_using_SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

https://wandb.ai/wandb/gallery/reports/SimpleTransformers-Transformers-Made-Easy--VmlldzoyNDQzNTg

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 5.6MB/s 
     |████████████████████████████████| 3.0MB 18.7MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 1.6MB 53.5MB/s 
     |████████████████████████████████| 7.2MB 51.5MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
     |████████████████████████████████| 317kB 53.8MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 163kB 59.3MB/s 
     |████████████████████████████████| 122kB 60.8MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 112kB 57.3MB/s 
     |████████████████████████████████| 4.4MB 41.3MB/s 
     |████████████████████████████████| 1.1MB 59.1MB/s 
     |████████████████████████████████| 890kB 53.3MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 122kB 64.5MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.1

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb

wandb.login()

# setup wandb environment variables
%env WANDB_ENTITY=wandb
%env WANDB_PROJECT=gallery

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_ENTITY=wandb
env: WANDB_PROJECT=gallery


In [ ]:
from simpletransformers.classification import ClassificationModel
import logging

import os
import sklearn
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Download and Prepare Dataset

#### Download StackOverflow dataset

Reference: https://www.tensorflow.org/tutorials/keras/text_classification#exercise_multiclass_classification_on_stack_overflow_questions

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz
!tar -xf stack_overflow_16k.tar.gz

--2020-09-18 10:36:59--  http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 74.125.199.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6053168 (5.8M) [application/x-gzip]
Saving to: ‘stack_overflow_16k.tar.gz’

stack_overflow_16k. 100%[===================>]   5.77M  23.7MB/s    in 0.2s    

2020-09-18 10:37:00 (23.7 MB/s) - ‘stack_overflow_16k.tar.gz’ saved [6053168/6053168]



#### Helper function to read and prepare dataset

In [ ]:
LABELS = ['csharp', 'java', 'javascript', 'python']

def read_imdb_split(data_dir):
    texts = []
    labels = []
    for i, label_dir in enumerate(LABELS):
        split_dir = Path(data_dir)
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(i)
    
    return texts, labels

train_texts, train_labels = read_imdb_split('train')
test_texts, test_labels = read_imdb_split('test')

#### Prepare train and validation split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

#### Train and Evaluation data needs to be in a Pandas Dataframe.

In [ ]:
train_data = list(zip(train_texts, train_labels))
test_data = list(zip(test_texts, test_labels))

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

#### Train and Evaluation data needs to be in a Pandas Dataframe.

# Fine-tune transformer model for binary text classification

Using SimpleTransformer it get's easier to experiment with different transformer. We just need three lines of code. 

#### Define train configs and define pretrained transformer model.

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 3,
    'train_batch_size': 16,
    'eval_batch_size': 64,
    'logging_steps': 10,
    'warmup_steps': 500,
    'weight_decay': 0.01,
    'wandb_project': 'gallery',
    "wandb_kwargs": {'entity': 'wandb'}
}

# Create a ClassificationModel
model = ClassificationModel('distilbert', 'distilbert-base-cased', num_labels=4, use_cuda=True, cuda_device=0, args=train_args)

INFO:filelock:Lock 140558084953312 acquired on /root/.cache/torch/transformers/774d52b0be7c2f621ac9e64708a8b80f22059f6d0e264e1bdc4f4d71c386c4ea.f44aaaab97e2ee0f8d9071a5cd694e19bf664237a92aea20ebe04ddb7097b494.lock


INFO:filelock:Lock 140558084953312 released on /root/.cache/torch/transformers/774d52b0be7c2f621ac9e64708a8b80f22059f6d0e264e1bdc4f4d71c386c4ea.f44aaaab97e2ee0f8d9071a5cd694e19bf664237a92aea20ebe04ddb7097b494.lock
INFO:filelock:Lock 140558084953592 acquired on /root/.cache/torch/transformers/185eb053d63bc5c2d6994e4b2a8e5eb59f31af90db9c5fae5e38c32a986462cb.857b7d17ad0bfaa2eec50caf481575bab1073303fef16bd5f29bc5248b2b8c7d.lock


INFO:filelock:Lock 140558084953592 released on /root/.cache/torch/transformers/185eb053d63bc5c2d6994e4b2a8e5eb59f31af90db9c5fae5e38c32a986462cb.857b7d17ad0bfaa2eec50caf481575bab1073303fef16bd5f29bc5248b2b8c7d.lock


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

INFO:filelock:Lock 140555604113168 released on /root/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


#### Easy to fine-tune transformer using `model.train_model`.

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:304: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: iit-bhu (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.1
wandb: Run data is saved locally in wandb/run-20200918_105052-1ly3jmix
wandb: Syncing run sandy-dust-27


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.


#### Finally evaluate the model and get classification report.

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:776: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


wandb: Waiting for W&B process to finish, PID 330
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200918_105052-1ly3jmix/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200918_105052-1ly3jmix/logs/debug-internal.log
wandb: Run summary:
wandb:                                                       Training loss 0.05137301981449127
wandb:                                                                  lr 0.0
wandb:                                                         global_step 1200
wandb:                                                               _step 119
wandb:                                                            _runtime 117
wandb:                                                          _timestamp 1600426370
wandb: Run history:
wandb:   Training loss ████▇▆▄▆▆▄▅▃▅▅▂▄▄▄▄▃▂▃▅▃▂▄▃▃▃▂▂▁▃▁▂▂▂▃▁▁
wandb:              lr ▁▂▂▂▃▃▄▄▅▅▅▆▆▇▇████▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▁▁
wandb:     global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7831670093023082, 'acc': 0.837375, 'eval_loss': 0.48627894085645673}
